In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from collections import Counter
from fuzzywuzzy import process, fuzz
import pickle
import requests
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import tqdm

C:\Users\Lin\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None

In [3]:
%%time
df = pd.read_csv("datos/datamarket_productos_de_supermercados.csv")
df.head(20)

CPU times: total: 12.4 s
Wall time: 12.4 s


,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id
0,https://www.dia.es/compra-online/productos/perfumeria-e-higiene/cuidado-del-cabello/champu/p/151922,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,FLEX champú con Keratina para cabello normal frasco 650 ml,NaN,3.39,5.22,l,2021-03-06 00:00:00,782a516202a4c7d4977341602ed10173
1,https://www.dia.es/compra-online/productos/perfumeria-e-higiene/cuidado-del-cabello/champu/p/152591,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,FLEX champú con Keratina fortificante frasco 650 ml,NaN,3.39,5.22,l,2021-03-06 00:00:00,fcadc0dcbdfaa1dc2aa6c99523c3fa47
2,https://www.dia.es/compra-online/productos/perfumeria-e-higiene/cuidado-del-cabello/champu/p/152271,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,GLISS champú reparador total frasco 250 ml,NaN,2.99,11.96,l,2021-03-06 00:00:00,bf86b25bafae2e646edba24ed9a0af7c
3,https://www.dia.es/compra-online/productos/perfumeria-e-higiene/cuidado-del-cabello/champu/p/152274,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,GLISS champú reparador ultimate frasco 250 ml,NaN,2.99,11.96,l,2021-03-06 00:00:00,df3b30f9e481ec851d606a99fd8ad8b7
4,https://www.dia.es/compra-online/productos/perfumeria-e-higiene/cuidado-del-cabello/champu/p/152175,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,NATUR VITAL champú protección color cabellos castaños bote 300 ml,NaN,3.99,13.30,l,2021-03-06 00:00:00,9c94c0754747ab0d37067ede61a682fc
5,https://www.dia.es/compra-online/productos/perfumeria-e-higiene/cuidado-del-cabello/champu/p/152176,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,NATUR VITAL champú sensitive hidratante con aloe vera bote 300 ml,NaN,3.99,13.30,l,2021-03-06 00:00:00,01a4e166feedfc4e791811275823d93f
6,https://www.dia.es/compra-online/productos/perfumeria-e-higiene/cuidado-del-cabello/champu/p/152177,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,NATUR VITAL champú sensitive cabellos grasos bote 300 ml,NaN,3.99,13.30,l,2021-03-06 00:00:00,a1657c0d723f8072848af4df9bd9ea3a
7,https://www.dia.es/compra-online/productos/perfumeria-e-higiene/cuidado-del-cabello/champu/p/152174,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,NATUR VITAL champú protección cabellos rubios bote 300 ml,NaN,3.99,13.30,l,2021-03-06 00:00:00,11c25e362a743664329964c829b7d182
8,https://www.dia.es/compra-online/productos/perfumeria-e-higiene/cuidado-del-cabello/champu/p/152179,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,NATUR VITAL champú anticaída cabellos grasos bote 300 ml,NaN,3.99,13.30,l,2021-03-06 00:00:00,1332c6b13329bbc90f2d419f7fdad10f
9,https://www.dia.es/compra-online/productos/perfumeria-e-higiene/cuidado-del-cabello/champu/p/152180,dia-es,perfumeria_e_higiene_cuidado_del_cabello_champu,NATUR VITAL champú anticaída cabello largo bote 300 ml,NaN,3.99,13.30,l,2021-03-06 00:00:00,174df037a80b67a36a14727bfe75a859


In [4]:
df.shape

(7843438, 10)

In [5]:
df.drop(["description"], axis = 1, inplace = True)

In [6]:
dfm = df[df["supermarket"] == "mercadona-es"]
dfc = df[df["supermarket"] == "carrefour-es"]
dfd = df[df["supermarket"] == "dia-es"]

In [7]:
# hay una categoria rara, la drop
dfd2 = dfd[dfd["category"] != "soy_solidario_soy_solidario_soy_solidario"]

In [8]:
"""
las categorías son horrendas, splitear y quedarme con algo no tiene sentido, porque el nombre más bien el tipo de producto no
sigue ningún tipo de patrón, es decir, al splitear, en algunos me tengo que quedar con el elemento 4, otro el último, penúltimo...
y pensando en coincidencias, fuzzybuzzy, el problemas es que el nombre del producto es bastante grande, por lo que
la coincidencia siempre sería bastante baja y muchas veces podría fallar, por lo que no es viable
lo que he pensado, es scrapear la info de nuevo, es decir, buscando el nombre del producto, creo que soy capaz de
conseguir de nuevo la info del género de los productos de una manera menos manual
pero cuando el producto no existe ya?
algunas urls no existen otras sí
en el buscador de Dia, no siempre es el primer objeto en la lísta de búsqueda por algún motivo
se puede automatizar haciendo que el nombre sea igual?

description solo lo tiene mercadona, es decir no me sirve
eliminar las horas, son ficticias y no dan info
una nueva columna con el peso litro, etc como tal??? cantidades que llevan los productos

huevos en mercadona están en docena en  vez de unidad
luego el supermercado sacar más info y sacarlo a otra tabla
el precio sacar también a otra tabla y que sea precio por fecha
product_id se repite, es decir, algunos productos tienen varios registros, pero no soy capaz de deplicarlo o generar uno nuevo
no sé de donde lo han sacado

tengo que saber muy bien qué scrapear, porque no quiero scrapear un producto y no ser capaz de correlacionarlo
pero quiero los productos nuevos??? sí??

he buscado más supermercados, pero la mayoría son mucho menores y tienen menos productos o tienen gran número de productos 
NO alimentíceos

las tablas, quiero tablas con más info de cada supermercado, quiero tablas que diga el precio del producto y de qué fecha se
ha tomado una posible tabla con la marca???? y datos de esta marca la marca la puedo sacar con las 
palabras con Mayus en merca y carrefour
en Dia está todo mayus

product id qué es buscarlo
"""

'\nlas categorías son horrendas, splitear y quedarme con algo no tiene sentido, porque el nombre más bien el tipo de producto no\nsigue ningún tipo de patrón, es decir, al splitear, en algunos me tengo que quedar con el elemento 4, otro el último, penúltimo...\ny pensando en coincidencias, fuzzybuzzy, el problemas es que el nombre del producto es bastante grande, por lo que\nla coincidencia siempre sería bastante baja y muchas veces podría fallar, por lo que no es viable\nlo que he pensado, es scrapear la info de nuevo, es decir, buscando el nombre del producto, creo que soy capaz de\nconseguir de nuevo la info del género de los productos de una manera menos manual\npero cuando el producto no existe ya?\nalgunas urls no existen otras sí\nen el buscador de Dia, no siempre es el primer objeto en la lísta de búsqueda por algún motivo\nse puede automatizar haciendo que el nombre sea igual?\n\ndescription solo lo tiene mercadona, es decir no me sirve\neliminar las horas, son ficticias y no 

In [9]:
dfd2["category"].value_counts()

despensa_lacteos_y_huevos_yogures                                                        48645
despensa_desayunos_y_dulces_galletas                                                     48499
despensa_desayunos_y_dulces_bolleria                                                     48495
cuidado_personal_bano_e_higiene_personal_cuidado_del_cabello                             48476
despensa_desayunos_y_dulces_caramelos_chicles_y_golosinas                                48426
despensa_desayunos_y_dulces_chocolates_y_bombones                                        48407
despensa_desayunos_y_dulces_cafe                                                         48012
cuidado_personal_cuidado_bucal                                                           47620
despensa_aperitivos                                                                      47509
cuidado_del_hogar_cuidado_de_la_ropa                                                     47455
bebidas_refrescos                                 

In [11]:
%%time
dfd2["subcategoria"] = dfd2.apply(lambda x: dia_cat(x["category"]), axis = 1)

CPU times: total: 33.8 s
Wall time: 33.9 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
dfd2["subcategoria"].value_counts()

conservas                     148211
cabello                       114673
colonias                       98946
quesos                         91708
precocinados                   91386
corporal                       78620
facial                         78285
gel_jabones                    72581
leche                          65489
refrescos                      59959
destilados_licores             58856
intimo                         58472
hogar                          58299
cervezas                       57305
ropa                           56926
bucal                          56044
aperitivos                     56042
yogures                        54568
bolleria                       54164
galletas                       53179
cafe                           51507
embutidos                      51244
zumos                          50512
ambientadores_insecticidas     49143
limpieza                       48895
depilacion                     48687
bombones                       48533
g

In [13]:
dfd2[dfd2["subcategoria"].isnull()]["category"].unique()

array([], dtype=object)

In [15]:
dfd2[dfd2["subcategoria"].isnull()]["category"].unique()

array([], dtype=object)

In [16]:
dfd2["subcategoria"].unique()


array(['cabello', 'vinos_tintos', 'higiene', 'destilados_licores',
       'cervezas', 'alimentacion', 'bucal', 'intimo',
       'ambientadores_insecticidas', 'ropa', 'lavavajillas', 'bano',
       'limpieza', 'hogar', 'colonias', 'depilacion', 'botiquin',
       'sangrias', 'cosmetica', 'vinos_blancos', 'vinos_rosados',
       'pan_tostado', 'pan', 'pescado', 'vegetales', 'hongos', 'fruta',
       'quesos', 'embutidos', 'ketchup', 'aperitivos', 'precocinados',
       'conservas', 'harinas', 'pastas', 'precocinados_cong', 'especias',
       'salsas', 'bolleria', 'cacao', 'cereales', 'chocolate', 'galletas',
       'dulces', 'mantequilla', 'otros', 'perros', 'gatos',
       'isotonicas_energeticas', 'refrescos', 'pescaderia_cong',
       'helados', 'huevos', 'yogures', 'leche', 'postres', 'batidos',
       'reposteria', 'cerdo', 'conejo_cordero', 'aves', 'infusiones',
       'dieteticos', 'panales', 'accesorios', 'cafe', 'cocina', 'calzado',
       'solar', 'facial', 'corporal', 'gel_jab

In [17]:
dfd2["subcategoria"].unique()

array(['cabello', 'vinos_tintos', 'higiene', 'destilados_licores',
       'cervezas', 'alimentacion', 'bucal', 'intimo',
       'ambientadores_insecticidas', 'ropa', 'lavavajillas', 'bano',
       'limpieza', 'hogar', 'colonias', 'depilacion', 'botiquin',
       'sangrias', 'cosmetica', 'vinos_blancos', 'vinos_rosados',
       'pan_tostado', 'pan', 'pescado', 'vegetales', 'hongos', 'fruta',
       'quesos', 'embutidos', 'ketchup', 'aperitivos', 'precocinados',
       'conservas', 'harinas', 'pastas', 'precocinados_cong', 'especias',
       'salsas', 'bolleria', 'cacao', 'cereales', 'chocolate', 'galletas',
       'dulces', 'mantequilla', 'otros', 'perros', 'gatos',
       'isotonicas_energeticas', 'refrescos', 'pescaderia_cong',
       'helados', 'huevos', 'yogures', 'leche', 'postres', 'batidos',
       'reposteria', 'cerdo', 'conejo_cordero', 'aves', 'infusiones',
       'dieteticos', 'panales', 'accesorios', 'cafe', 'cocina', 'calzado',
       'solar', 'facial', 'corporal', 'gel_jab

In [18]:
%%time
dfd2["categoria"] = dfd2.apply(lambda x: categoryb(x["subcategoria"]), axis = 1)

CPU times: total: 18.4 s
Wall time: 18.4 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
dfd2["categoria"].unique()

array(['cuidado_higiene_personal', 'alcoholicas', 'bebe',
       'cuidado_higiene_hogar', 'pan', 'pescaderia', 'verduleria', None,
       'carniceria', 'despensa', 'snacks', 'precocinados', 'congelados',
       'desayunos', 'huevos_lacteos_derivados', 'mascotas', 'bebidas',
       'internacional'], dtype=object)

In [43]:
def mer_cat(categoria):
    frag = categoria.split("_")
    dicmercuid = {"zumos": "zumos", "fitoterapia": "botiquin", "pizzas": "precocinados",
           "aperitivos": "aperitivos", "maquillaje": "cosmetica"}
    dicmercon = {"helados": ["helados"],
             "vegetales_cong": ["verdura"], 
             "pescaderia_cong": ["pescado", "marisco"],
             "precocinados_cong": ["churros", "rebozados", "pizzas", "pasta"],
             "hielo": ["hielo"],
             "carnes_cong" : ["carne"]}
    dicmercharc = {"queso" : ["fresco", "curado", "tierno", "porciones"],
               "embutidos": ["cocido", "serrano", "sobrasada", "salchichas", "mortadela"]}
    dicmerpan = {"pan": ["horno"],
              "bolleria": ["envasada", "pasteles"],
              "harinas": ["reposteria"],
              "molde": ["especialidades"],
              "rallado" : ["rallado"],
              "pan_tostado": ["picatostes"],
              "reposteria": ["decoracion"]}
    dicmercui = {"colonias": ["colonia", "desodorante"],
             "cabello": ["cabello", "mascarilla", "accesorios", "champu"],
             "intimo": ["intima"],
             "facial": ["facial"],
             "bucal": ["bucal"],
             "corporal": ["corporal", "pedicura", "neceseres"],
             "gel_jabones": ["manos"],
             "depilacion": ["hombre", "depilacion"],
             "solar": ["aftersun"]}
    dicmerhue = {"leche": ["vegetales"],
             "mantequilla": ["margarina"],
             "huevos" : ["huevos"]}
    dicmerazuc = {"chocolate": ["chocolate"],
              "dulces": ["caramelos", "turrones"],
              "conservas": ["miel"],
              "golosinas": ["golosinas"],
              "edulcorantes": ["edulcorante"]}
    dicmerbod = {"cervezas": ["cerveza", "cava", "alcohol"],
             "destilados_licores": ["licores", "espumoso"],
             "vinos_tintos": ["tinto"],
             "vinos_blancos": ["blanco"],
             "vinos_rosados": ["rosado"],
             "sangrias": ["sangria"]}
    dicmerfrut = {"vegetales": ["verdura"],
              "fruta": ["fruta"],
              "precocinados": ["preparada"]}
    dicmerlim = {"ambientadores_insecticidas": ["ambientador"],
             "calzado": ["calzado"],
             "ropa": ["ropa"],
             "cocina":["alimentos", "cocina"],
             "hogar": ["guantes", "basura", "celulosa"],
             "limpieza": ["fuertes", "multiusos", "friegasuelos", "limpiacristales"],
             "bano": ["wc"],
             "lavavajillas": ["vajilla"]}
    dicmercer = {"galletas": ["galletas", "tortitas"],
             "cereales": ["cereales"]}
    dimercarr = {"pastas": ["fideos"],
             "legumbres": ["legumbres"],
             "arroz": ["arroz"]}
    dicmerbebe = {"alimentacion": ["infantil", "menaje"],
              "panales": ["panales"],
              "higiene" : ["cuidado"]}
    dicmercons = {"conservas": ["frutas", "pescado", "mejillones"],
                   "precocinados": ["caldo", "cremas", "tomate"]}
    dicmermasc = {"perros": ["perro"],
              "gatos": ["gato"],
              "otros": ["otros"]}
    dicmercar = {"aves": ["pollo"],
             "cerdo": ["cerdo"],
             "picados": ["picadas"],
             "vacuno": ["vacuno"],
             "elaborados": ["elaborados"],
             "carnes_cong": ["congeladas", "congelada"],
             "embutidos": ["embutidos", "arreglos", "embutido"],
             "conejo_cordero" : ["cordero"]}
    dicmermari = {"pescado" : ["fresco", "bandeja", "ahumados"],
              "precocinados": ["sushi"],
              "marisco": ["marisco"],
              "pescaderia_cong": ["congelado"]}
    dicmeracei = {"especias": ["especias"],
              "salsas": ["salsas"],
              "raro" : ["sal", "mostaza"]}
    dicmeragua = {"agua": ["agua"],
              "refrescos": ["cola", "limon", "gas", "bitter"],
              "isotonicas_energeticas": ["energetico"]}

    dicmercaca = {"cafe": ["monodosis", "bebidas", "grano"],
              "infusiones": ["infusiones"],
              "cacao": ["taza"]}
    dicmerpost = {"yogures": ["liquidos", "bifidus", "sabores", "desnatados", "griegos", "infantiles"],
              "postres": ["postres", "natillas", "soja"]}
    dic1 = {"congelados": dicmercon, "charcuteria": dicmercharc}
    dic2 = {"cuidado": dicmercui, "huevos": dicmerhue, "azucar": dicmerazuc, "bodega": dicmerbod, "fruta": dicmerfrut, "limpieza": dicmerlim,
           "cereales": dicmercer, "arroz": dicmerarr, "bebe": dicmerbebe, "conservas": dicmercons, "mascotas": dicmermasc,
           "carne": dicmercar, "marisco": dicmermari, "aceite": dicmeracei, "agua": dicmeragua, "cacao": dicmercaca, "postres": dicmerpost}
    if frag[0] in diccuid:
        return diccuid[frag[0]]
    elif frag[0] in ["congelados", "charcuteria"]:
        for k, v in dic1[frag[0]].items():
            if frag[-1] in v:
                return k
    elif frag[0] == "panaderia":
        if frag[-1] == "horno":
            return frag[-3]
        else:
            for k, v in dicpan.items():
                if frag[-1] in v:
                    return k
    else:
        for k, v in dic2[frag[0]].items():
            if frag[-1] in v:
                return k

In [44]:
%%time
dfm["subcategoria"] = dfm.apply(lambda x: mer_cat(x["category"]), axis = 1)

CPU times: total: 27.5 s
Wall time: 27.6 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [45]:
dfm[dfm["subcategoria"].isnull()]["category"].unique()

array([], dtype=object)

In [46]:
%%time
dfm["categoria"] = dfm.apply(lambda x: categoryb(x["subcategoria"]), axis = 1)

CPU times: total: 18.6 s
Wall time: 18.8 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [47]:
dfm["categoria"].unique()

array(['huevos_lacteos_derivados', 'cuidado_higiene_hogar', 'bebidas',
       'congelados', 'alcoholicas', 'desayunos',
       'cuidado_higiene_personal', 'snacks', 'despensa', 'verduleria',
       'precocinados', 'carniceria', 'bebe', 'pescaderia', 'mascotas',
       'pan'], dtype=object)